# Task 1 d, Calculating strength reserve factor of all elements

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = "fabian"

## Imports 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *


## File Selection 

In [3]:
df = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
print(df)

           Component Name  sigmaXX  sigmaYY  sigmaZZ  sigmaXY  sigmaXZ  \
Element ID                                                               
1                  panel1   -66.67   12.750        0    26.25        0   
2                  panel1   -82.97   20.870        0    27.10        0   
3                  panel1   -67.63   20.740        0    36.97        0   
4                  panel2   -97.67    2.415        0    34.52        0   
5                  panel2   -89.50   25.350        0    27.04        0   
...                   ...      ...      ...      ...      ...      ...   
26                 panel9   -44.75   12.670        0   162.30        0   
27                 panel9   -52.64  -65.300        0   190.50        0   
28                panel10   -36.45   32.350        0   192.30        0   
29                panel10   -41.48   10.440        0   162.60        0   
30                panel10   -30.70  -15.600        0   187.00        0   

            sigmaYZ        Load Case 

## Constants 

In [4]:
sigma_ult = 560

## Applying Functions 

In [ ]:
df['reserveFactor'] = df.apply(strength_calc, sigma_ult= sigma_ult, axis=1)
df

,Component Name,sigmaXX,sigmaYY,sigmaZZ,sigmaXY,sigmaXZ,sigmaYZ,Load Case,reserveFactor
Element ID,,,,,,,,,
1,panel1,-66.67,12.750,0,26.25,0,0,Subcase 1 (LC1),4.304
2,panel1,-82.97,20.870,0,27.10,0,0,Subcase 1 (LC1),3.519
3,panel1,-67.63,20.740,0,36.97,0,0,Subcase 1 (LC1),3.642
4,panel2,-97.67,2.415,0,34.52,0,0,Subcase 1 (LC1),3.230
5,panel2,-89.50,25.350,0,27.04,0,0,Subcase 1 (LC1),3.260
...,...,...,...,...,...,...,...,...,...
26,panel9,-44.75,12.670,0,162.30,0,0,Subcase 3 (LC3),1.306
27,panel9,-52.64,-65.300,0,190.50,0,0,Subcase 3 (LC3),1.113
28,panel10,-36.45,32.350,0,192.30,0,0,Subcase 3 (LC3),1.103


## Export back to Excel 